In [1]:
import torch
import torch.nn.functional as F
import torch.optim as optim
import torch.nn as nn
from models import MLP, NaiveCNN, RNN
from utils import get_dataset
from torchvision import datasets, transforms

# FMNIST

In [2]:
lr = 5e-3
reg = 1e-4
EPOCHS=20
batch_size=128
MOMENTUM = 0.9

In [3]:
apply_transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))])

train_dataset = datasets.FashionMNIST('./../data', train=True, download=True,transform=apply_transform)
test_dataset = datasets.FashionMNIST('./../data', train=False, download=True,transform=apply_transform)
trainloader = torch.utils.data.DataLoader(train_dataset,batch_size = batch_size, shuffle=True,num_workers=1)
testloader = torch.utils.data.DataLoader(test_dataset,batch_size = 128, shuffle=False,num_workers=1)

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
net = MLP(784,[64,30],10)
net = net.to(device)
if device =='cuda':
    print("Train on GPU...")
else:
    print("Train on CPU...")
criterion = F.cross_entropy
# Your code: use an optimizer
optimizer = optim.SGD(net.parameters(), lr=lr,momentum=MOMENTUM,weight_decay=reg)

Train on GPU...


# CIFAR-10

In [2]:
lr = 0.01
reg = 3e-4
EPOCHS=20
batch_size=128
MOMENTUM = 0.9

In [3]:
apply_transform = transforms.Compose(
            [transforms.ToTensor(),
             transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_dataset = datasets.CIFAR10('./../data', train=True, download=True,transform=apply_transform)

test_dataset = datasets.CIFAR10('./../data', train=False, download=True,transform=apply_transform)
trainloader = torch.utils.data.DataLoader(train_dataset,batch_size = batch_size, shuffle=True,num_workers=1)
testloader = torch.utils.data.DataLoader(test_dataset,batch_size = 128, shuffle=False,num_workers=1)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
class NaiveCNN(nn.Module):
    def __init__(self):
        super(NaiveCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3)
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.conv3 = nn.Conv2d(64,64,3)
        self.fc1   = nn.Linear(4*4*64, 64)
        self.fc2   = nn.Linear(64, 10)
        self.activation = nn.ReLU()

    def forward(self, x):
        out = self.activation(F.max_pool2d(self.conv1(x),2))
        out = self.activation(F.max_pool2d(self.conv2(out),2))
        out = self.activation(self.conv3(out))
        out = out.view(out.size(0), -1)
        out = self.activation(self.fc1(out))
        out = self.fc2(out)
        return out

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
net = NaiveCNN()
net = net.to(device)
if device =='cuda':
    print("Train on GPU...")
else:
    print("Train on CPU...")
criterion = F.cross_entropy
# Your code: use an optimizer
optimizer = optim.SGD(net.parameters(), lr=lr,momentum=MOMENTUM,weight_decay=reg)

Train on GPU...


In [6]:
global_step = 0
best_val_acc = 0

val_acc_summary = []
train_loss_summary = []
for i in range(EPOCHS):
    # Switch to train mode
    net.train()
    print("Epoch %d:" %i)

    total_examples = 0
    correct_examples = 0

    train_loss = 0
    train_acc = 0
    
    # Train the training dataset for 1 epoch.
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        # Your code: Copy inputs to device
        inputs = inputs.to(device)
        targets = targets.to(device).long()
        # Your code: Zero the gradient of the optimizer (1 Line)
        net.zero_grad()
        # Your code: Generate output (1 Line)
        outputs = net(inputs)
        # Your code: Compute loss (1 Line)
        loss = criterion(outputs,targets)
        # Your code: Now backward loss and compute gradient (1 Line)
        loss.backward()
        # Your code: apply gradient (1 Line)
        optimizer.step()
        # Calculate predicted labels
        _, predicted = outputs.max(1)
        total_examples += predicted.size(0)
        correct_examples += predicted.eq(targets).sum().item()
        train_loss += loss
        global_step += 1
        if i==0 and batch_idx==0:
            print("Initial Loss: %.4f"%loss)
                
    avg_loss = train_loss / (batch_idx + 1)
    avg_acc = correct_examples / total_examples
    print("Training loss: %.4f, Training accuracy: %.4f" %(avg_loss, avg_acc))
    
    train_loss_summary.append(avg_loss)
    # Validate on the validation dataset
    print("Test...")
    total_examples = 0
    correct_examples = 0
    
    net.eval()


    val_loss = 0
    val_acc = 0
    # Disable gradient during validation
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            # Copy inputs to device
            inputs = inputs.to(device)
            targets = targets.to(device).long()
            # Zero the gradient
            optimizer.zero_grad()
            # Generate output from the DNN.
            outputs = net(inputs)
            loss = criterion(outputs, targets)            
            # Calculate predicted labels
            _, predicted = outputs.max(1)
            total_examples += predicted.size(0)
            correct_examples += predicted.eq(targets).sum().item()
            val_loss += loss

    avg_loss = val_loss / len(testloader)
    avg_acc = correct_examples / total_examples
    
    print("Test loss: %.4f, Test accuracy: %.4f" % (avg_loss, avg_acc))
    val_acc_summary.append(avg_acc)
    # # Handle the learning rate scheduler.
    # if i in DECAY_EPOCHS:
    #     current_learning_rate = current_learning_rate * DECAY
    #     for param_group in optimizer.param_groups:
    #         param_group['lr'] = current_learning_rate
    #     print("Current learning rate has decayed to %f" %current_learning_rate)
    
    # Save for checkpoint
    if avg_acc > best_val_acc:
        best_val_acc = avg_acc
    #     if not os.path.exists(CHECKPOINT_PATH):
    #         os.makedirs(CHECKPOINT_PATH)
    #     print("Saving ...")
    #     state = {'net': net.state_dict(),
    #              'epoch': i,
    #              'lr': current_learning_rate}
    #     torch.save(state, os.path.join(CHECKPOINT_PATH, 'model.h5'))
    # if i==29:
    #     print("Best Accuraacy after 30 epochs: %.4f"%best_val_acc)

print("Optimization finished.")
print("Best Accuracy: %.4f"%best_val_acc)

Epoch 0:
Initial Loss: 2.3025
Training loss: 1.8887, Training accuracy: 0.3037
Test...
Test loss: 1.5885, Test accuracy: 0.4118
Epoch 1:
Training loss: 1.4499, Training accuracy: 0.4715
Test...
Test loss: 1.3799, Test accuracy: 0.5027
Epoch 2:
Training loss: 1.2572, Training accuracy: 0.5489
Test...
Test loss: 1.1834, Test accuracy: 0.5730
Epoch 3:
Training loss: 1.1044, Training accuracy: 0.6092
Test...
Test loss: 1.1429, Test accuracy: 0.5939
Epoch 4:
Training loss: 0.9901, Training accuracy: 0.6499
Test...
Test loss: 1.0119, Test accuracy: 0.6466
Epoch 5:
Training loss: 0.9016, Training accuracy: 0.6819
Test...
Test loss: 0.9159, Test accuracy: 0.6856
Epoch 6:
Training loss: 0.8237, Training accuracy: 0.7121
Test...
Test loss: 0.9101, Test accuracy: 0.6832
Epoch 7:
Training loss: 0.7660, Training accuracy: 0.7306
Test...
Test loss: 0.8289, Test accuracy: 0.7087
Epoch 8:
Training loss: 0.7182, Training accuracy: 0.7485
Test...
Test loss: 0.7947, Test accuracy: 0.7255
Epoch 9:
Trainin